In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score

### 1. importing dataset
* removed observations where IsBadBuy is missing

In [2]:
df = pd.read_csv('kickcars-sample-200.csv')
df = df[df['IsBadBuy'].notnull()]
df.shape

(195, 34)

### 2. removing columns
* dropped PurchDate and VehYear as we already have VehicleAge
* dropped TopThreeAmericanName as as we already have Make
* dropped WheelTypeID as we already have WheelType
* dropped VNZIP1 as we already have VNST
* dropped Model and SubModel as there are multiple values
* dropped Trim as there are multiple values
* dropped Auction and IsOnlineSale as there are single values
* dropped Color and BYRNO as these may be misleading
* dropped AUCGUART as it is same as PRIMEUNIT

In [3]:
df = df.drop(['PurchDate','VehYear','WheelTypeID','TopThreeAmericanName','VNZIP1'], axis=1)
df = df.drop(['Model','SubModel','Trim'], axis=1)
df = df.drop(['Auction','IsOnlineSale','Color','BYRNO','AUCGUART'], axis=1)
df.shape

(195, 21)

### 3. creating columns - numerical
* ratio of acquisition cost paid for the vehicle at time of purchase is taken
* if ratio is less than or equal to zero then it is made missing
* missing value is also imputed
* all positively related

In [4]:
df['MMRAcquisitionAuctionAveragePrice'] = df['VehBCost'] / df['MMRAcquisitionAuctionAveragePrice']
df['MMRAcquisitionAuctionCleanPrice'] = df['VehBCost'] / df['MMRAcquisitionAuctionCleanPrice']
df['MMRAcquisitionRetailAveragePrice'] = df['VehBCost'] / df['MMRAcquisitionRetailAveragePrice']
df['MMRAcquisitonRetailCleanPrice'] = df['VehBCost'] / df['MMRAcquisitonRetailCleanPrice']
df['MMRCurrentAuctionAveragePrice'] = df['VehBCost'] / df['MMRCurrentAuctionAveragePrice']
df['MMRCurrentAuctionCleanPrice'] = df['VehBCost'] / df['MMRCurrentAuctionCleanPrice']
df['MMRCurrentRetailAveragePrice'] = df['VehBCost'] / df['MMRCurrentRetailAveragePrice']
df['MMRCurrentRetailCleanPrice'] = df['VehBCost'] / df['MMRCurrentRetailCleanPrice']
df['WarrantyCost'] = df['WarrantyCost'] / df['VehBCost']

df['MMRAcquisitionAuctionAveragePrice'] = np.where(df['MMRAcquisitionAuctionAveragePrice']==np.inf, np.nan, df['MMRAcquisitionAuctionAveragePrice'])
df['MMRAcquisitionAuctionCleanPrice'] = np.where(df['MMRAcquisitionAuctionCleanPrice']==np.inf, np.nan, df['MMRAcquisitionAuctionCleanPrice'])
df['MMRAcquisitionRetailAveragePrice'] = np.where(df['MMRAcquisitionRetailAveragePrice']==np.inf, np.nan, df['MMRAcquisitionRetailAveragePrice'])
df['MMRAcquisitonRetailCleanPrice'] = np.where(df['MMRAcquisitonRetailCleanPrice']==np.inf, np.nan, df['MMRAcquisitonRetailCleanPrice'])
df['MMRCurrentAuctionAveragePrice'] = np.where(df['MMRCurrentAuctionAveragePrice']==np.inf, np.nan, df['MMRCurrentAuctionAveragePrice'])
df['MMRCurrentAuctionCleanPrice'] = np.where(df['MMRCurrentAuctionCleanPrice']==np.inf, np.nan, df['MMRCurrentAuctionCleanPrice'])
df['MMRCurrentRetailAveragePrice'] = np.where(df['MMRCurrentRetailAveragePrice']==np.inf, np.nan, df['MMRCurrentRetailAveragePrice'])
df['MMRCurrentRetailCleanPrice'] = np.where(df['MMRCurrentRetailCleanPrice']==np.inf, np.nan, df['MMRCurrentRetailCleanPrice'])
df['WarrantyCost'] = np.where(df['WarrantyCost']==np.inf, np.nan, df['WarrantyCost'])

df['MMRAcquisitionAuctionAveragePrice'] = df['MMRAcquisitionAuctionAveragePrice'].fillna(df['MMRAcquisitionAuctionAveragePrice'].mean())
df['MMRAcquisitionAuctionCleanPrice'] = df['MMRAcquisitionAuctionCleanPrice'].fillna(df['MMRAcquisitionAuctionCleanPrice'].mean())
df['MMRAcquisitionRetailAveragePrice'] = df['MMRAcquisitionRetailAveragePrice'].fillna(df['MMRAcquisitionRetailAveragePrice'].mean())
df['MMRAcquisitonRetailCleanPrice'] = df['MMRAcquisitonRetailCleanPrice'].fillna(df['MMRAcquisitonRetailCleanPrice'].mean())
df['MMRCurrentAuctionAveragePrice'] = df['MMRCurrentAuctionAveragePrice'].fillna(df['MMRCurrentAuctionAveragePrice'].mean())
df['MMRCurrentAuctionCleanPrice'] = df['MMRCurrentAuctionCleanPrice'].fillna(df['MMRCurrentAuctionCleanPrice'].mean())
df['MMRCurrentRetailAveragePrice'] = df['MMRCurrentRetailAveragePrice'].fillna(df['MMRCurrentRetailAveragePrice'].mean())
df['MMRCurrentRetailCleanPrice'] = df['MMRCurrentRetailCleanPrice'].fillna(df['MMRCurrentRetailCleanPrice'].mean())
df['WarrantyCost'] = df['WarrantyCost'].fillna(df['WarrantyCost'].mean())

df['VehicleAge_decades'] = df['VehicleAge']/10
df['VehOdo_lakhs'] = df['VehOdo']/100000
df.shape

(195, 23)

### 4. creating columns - categorical
* converting categorical to numerical - Make, Transmission, WheelType, Nationality, Size, PRIMEUNIT, AUCGUART, VNST
* all are positively related - this is based on the imputation

In [5]:
df['Make'] = np.where(df['Make'].isin(['FORD','PONTIAC','SATURN']), 1, 
                      np.where(df['Make'].isin(['DODGE']), -1, 0))
df['Transmission'] = np.where(df['Transmission']=='AUTO', 1, -1)
df['WheelType'] = np.where(df['WheelType'].isnull(), 1, -1)
df['Nationality'] = np.where(df['Nationality']=='TOP LINE ASIAN', 1, -1)
df['Size'] = np.where(df['Size'].isin(['MEDIUM','COMPACT','LARGE','VAN']), 0, 
                      np.where(df['Size'].isin(['MEDIUM SUV']), 1, -1))
df['PRIMEUNIT'] = np.where(df['PRIMEUNIT'].isnull(), 1, -1)
df['VNST'] = np.where(df['VNST']=='VA', 1, -1)
df.shape

(195, 23)

### 5. exporting dataset
* there are 11 numerical variables
* there are 7 categorical variables
* there are 2 other variables - RefId and IsBadBuy	
* also looking at univariate

In [6]:
df = df.drop(['VehicleAge','VehOdo','VehBCost'], axis=1)
df.to_csv('01 data prep.csv', index=False)
df.shape

(195, 20)

In [7]:
df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
RefId,195.0,250.548718,141.320070,16.000000,128.500000,253.000000,374.500000,481.000000
IsBadBuy,195.0,0.312821,0.464835,0.000000,0.000000,0.000000,1.000000,1.000000
Make,195.0,0.148718,0.628725,-1.000000,0.000000,0.000000,1.000000,1.000000
Transmission,195.0,0.856410,0.517625,-1.000000,1.000000,1.000000,1.000000,1.000000
WheelType,195.0,-0.794872,0.608339,-1.000000,-1.000000,-1.000000,-1.000000,1.000000
Nationality,195.0,-0.897436,0.442280,-1.000000,-1.000000,-1.000000,-1.000000,1.000000
Size,195.0,-0.102564,0.555890,-1.000000,0.000000,0.000000,0.000000,1.000000
MMRAcquisitionAuctionAveragePrice,195.0,1.246107,0.312993,0.751520,1.011608,1.205597,1.422451,2.516779
MMRAcquisitionAuctionCleanPrice,195.0,1.001423,0.204953,0.646927,0.837864,0.975627,1.128049,1.647954
MMRAcquisitionRetailAveragePrice,195.0,0.776741,0.161716,0.516369,0.663671,0.756496,0.855805,1.404270


In [8]:
df.drop('RefId', axis=1).corr()

,IsBadBuy,Make,Transmission,WheelType,Nationality,Size,MMRAcquisitionAuctionAveragePrice,MMRAcquisitionAuctionCleanPrice,MMRAcquisitionRetailAveragePrice,MMRAcquisitonRetailCleanPrice,MMRCurrentAuctionAveragePrice,MMRCurrentAuctionCleanPrice,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,VNST,WarrantyCost,VehicleAge_decades,VehOdo_lakhs
IsBadBuy,1.000000,0.228022,0.187645,0.464596,0.043717,0.104857,0.076250,0.055340,0.202481,0.176905,0.033969,0.008425,0.178147,0.121945,0.161416,0.264113,0.093795,0.185157,0.061162
Make,0.228022,1.000000,-0.187467,0.054599,-0.055136,-0.029875,0.077252,0.051625,-0.039140,-0.077145,0.074869,0.050598,-0.032803,-0.057658,0.100698,-0.055790,0.097128,0.208164,0.110047
Transmission,0.187645,-0.187467,1.000000,0.094020,0.064660,0.020211,-0.079798,-0.060547,0.035218,0.064300,-0.075611,-0.061242,0.087562,0.098185,-0.083149,0.063530,0.018193,-0.040668,0.005132
WheelType,0.464596,0.054599,0.094020,1.000000,0.151301,0.062534,-0.050222,-0.062474,0.116064,0.089371,-0.037156,-0.044596,0.131731,0.078998,0.101072,0.183791,-0.068868,0.075193,0.042646
Nationality,0.043717,-0.055136,0.064660,0.151301,1.000000,0.084938,-0.012597,0.026484,0.008320,0.020578,-0.044319,-0.009258,-0.007187,0.002607,-0.015205,-0.034698,-0.165877,0.097400,0.070197
Size,0.104857,-0.029875,0.020211,0.062534,0.084938,1.000000,0.016773,-0.019343,0.016703,0.022765,0.110201,0.067442,0.061186,0.027564,-0.156407,0.078875,0.181911,-0.015179,0.035328
MMRAcquisitionAuctionAveragePrice,0.076250,0.077252,-0.079798,-0.050222,-0.012597,0.016773,1.000000,0.965074,0.621714,0.567897,0.811703,0.779849,0.520665,0.495718,-0.109826,-0.066493,0.288463,0.713065,-0.012699
MMRAcquisitionAuctionCleanPrice,0.055340,0.051625,-0.060547,-0.062474,0.026484,-0.019343,0.965074,1.000000,0.630122,0.627431,0.770470,0.784028,0.524392,0.531132,-0.122149,-0.114179,0.193390,0.668135,-0.066893
MMRAcquisitionRetailAveragePrice,0.202481,-0.039140,0.035218,0.116064,0.008320,0.016703,0.621714,0.630122,1.000000,0.964086,0.516961,0.531651,0.873097,0.852501,0.007992,0.561585,0.121564,0.386631,-0.122051
MMRAcquisitonRetailCleanPrice,0.176905,-0.077145,0.064300,0.089371,0.020578,0.022765,0.567897,0.627431,0.964086,1.000000,0.454126,0.503962,0.832747,0.849408,-0.011028,0.495682,0.032320,0.308321,-0.180240
